# Dead Run RHESSys Workflow
<br />
RHESSysWorkflows provides a series of Python tools for performing [RHESSys](https://github.com/RHESSys/RHESSys) data preparation workflows. These tools build on the workflow system defined by [EcohydroLib](https://github.com/selimnairb/EcohydroLib) and [RHESSysWorkflows](https://github.com/selimnairb/RHESSysWorkflows).
<br />
### This notebook uses custom datasets and is for advanced users comfortable with a linux terminal and using text editor such as Vi.
<br />
Dead Run, USGS gage [01589330](http://waterdata.usgs.gov/usa/nwis/uv?01589330) is located at Franklintown MD.
<br />

### The general steps to use RHESSys workflows 
1 Register DEM <br />
2 Import Gage <br />
3 Download soil data<br />
4 Prepare Land Cover data<br />
5 Download LAI data<br />
6 Create a new GRASS GIS Location<br />
7 Import RHESSys code and compile (automatically or manually)<br />
8 Import Climate data<br />
9 Delineate watershed <br />
10 Generate Patch map <br />
11 Process soil maps <br />
12 Generate derived landcover maps<br />
13 Generate Rules and Reclassify <br />
14 Generate template <br />
15 Create world <br />
16 Create flow table <br />
17 Initializing vegetation carbon and nitrogen stores <br />
18 Creating a RHESSys TEC file <br />
19 Running RHESSys models <br />

This notebook is built on **RHESSys sample workflow** and **RHESSys Workflow at Coweeta, NC** examples. Not all steps are documented. Here we focus on explaining new or modifications. <br/>
<br/>
Users interested in seeing step outputs, remove **output = ** from the command line. Ouput and messages are also available in the log file.

In [ ]:
import os
import logging
import hs_utils
import shutil
from rhessys_wf import *
%matplotlib inline

In [ ]:
w = RHESSysWorkflow(project_name='dead_run_testV2', 
                    gageid='custom',
                    start_date='2008-01-01',
                    end_date='2010-01-01'
                    )

In [ ]:
hs = hs_utils.hydroshare()

### Please note: This resource is large and retrieving may take a few minutes.

In [ ]:
resource_id = '6dbb0dfb8f3a498881e4de428cb1587c'
# get ETV data bundle for this tutotial stored on HydroShare
content = hs.getResourceFromHydroShare(resource_id)

### Please note: This resource is large and unzipping may take a few minutes.

In [ ]:
zipfolder = w.sub_project_folder + '/RHESSys_ETV'
w.create_path(zipfolder)
zipfilepathname = hs.content['DR5_3m_nonburned_DEM_rain_duration_DEM_float_lctest_raingarden.zip']
w.unzip_etv_zip_file_at_path(zipfilepathname, zipfolder)
print 'Unzip Finished'

## Step 1 DEM

From the HydroShare resource downloaded above, a DEM is provided for DeadRun. The DEM.tif is location in the **zipfolder** location. If you are using a different resource, you will need to change the **datasource_folder** and  **demfile** values below. These changes are required with other data steps below. <br/><br/>
The **os.environ["GDAL_DATA"]** indicates to PYTHON where the GDAL installation is. If you have compiled GDAL (i.e. for specialized libraries) in your environment, change this value.
<br/><br/>

**RegisterDEM** will result in the DEM being copied to your project directory, and the DEM spatial resolution will determine the default spatial resolution of your project (i.e. other rasters imported to your project will by default be resampled to the match the DEM resolution and spatial reference). Also, the DEM extent will be used to determine the bounding box for the study area; a polygon of the DEM extent will be generated and saved as a shapefile in your project directory.

In [ ]:
os.environ["GDAL_DATA"] = "/opt/conda/share/gdal"
datasource_folder = '/RHESSys_ETV/DR5_3m_nonburned_DEM_rain_duration_DEM_float_lctest_raingarden/'
demfile = w.sub_project_folder + datasource_folder + 'DEM.tif'
#print(demfile)
w.RegisterDEM(w.sub_project_folder, demfile)

## Step 2 Import Gage
We will use the **RegisterGage** tool to tell EcohydroLib the coordinates of the Dead Run streamflow gage. This tool takes as input a point shapefile containing one or more points; the WGS84 lat-lon coordinates for the desired gage will be extracted from the shapefile. These coordinates will be written to the metadata store, and a new point shapefile containing a point only for the selected gage will be created in the project directory. <br/><br/>
You will need to determine the **shape field** with the **id_attribute**. You can use GIS products from GDAL, ESRI, QGIS to determine these values.


In [ ]:
gagefile = w.sub_project_folder + '/RHESSys_ETV/DR5_3m_nonburned_DEM_rain_duration_DEM_float_lctest_raingarden/gage.shp'
layername = 'gage'
id_attribute = 'gage_id'
id_value = '01589312'

print(gagefile)
w.RegisterGage(w.sub_project_folder, gagefile, layername, id_attribute, id_value)

## Step 3 Download Soil

The USDA NRCS provides the [Soil Data Mart](http://sdmdataaccess.nrcs.usda.gov/), a web services-based interface for querying and downloading high-resolution SSURGO soils data.

In [ ]:
output = w.get_SSURGOFeaturesForBoundingbox(w.sub_project_folder)
output = w.GenerateSoilPropertyRastersFromSSURGO(w.sub_project_folder)

## Step 4 Prepare Landcover Data
From the HydroShare resource downloaded above, a Landcover raster dataset is provided for DeadRun. The landcover.tif is location in the **zipfolder** location. If you are using a different resource, you will need to change the **datasource_folder** and **landcover.tif** values below. <br/><br/>

In [ ]:
landcover_fullpathname_Src = w.sub_project_folder + datasource_folder + 'landcover.tif'
landcover_fullpathname_Dec = w.sub_project_folder + '/landcover_test.tif'
shutil.copy(landcover_fullpathname_Src, landcover_fullpathname_Dec)

We'll use the **RegisterRaster** command to import landcover data into our project. By default, RegisterRaster will resample the raster it is importing (landcover in this case) to match the spatial reference and resolution of the DEM.

In [ ]:
landcover_fullpathname = w.sub_project_folder + '/landcover_test.tif'
options = " -p " + w.sub_project_folder + " -t landcover -r " + landcover_fullpathname + " --force"
#print(options)
w.RegisterRasterOptions(options)

## Step 5 Prepare LAI Data

In [ ]:
lai_fullpathname_Src = w.sub_project_folder + datasource_folder + 'lai.tif'
lai_fullpathname_Dec = w.sub_project_folder + '/lai_test.tif'
shutil.copy(lai_fullpathname_Src, lai_fullpathname_Dec)

options = " -p " + w.sub_project_folder + " -t lai -r " + lai_fullpathname_Dec + " --force"
#print(options)
w.RegisterRasterOptions(options)

## Step 6 Create a new GRASS GIS Location

RHESSys requires that all spatial data used to create a world file and flow table for a RHESSys model be stored in a GRASS GIS mapset. We'll start building these data in RHESSysWorkflows by creating a new GRASS location and mapset within our project directory, and importing our DEM into this mapset using the **CreateGRASSLocationFromDEM** tool. <br/>
Note: Output from CreateGRASSLocationFromDEM has been piped to output variable. For debugging, uncomment print statement (i.e. remove #).

In [ ]:
output = w.CreateGRASSLocationFromDEM(w.sub_project_folder, '"Dead Run 5 near Catonsville, 3m model"')
print(output)

## Step 7 Import RHESSys Code
To create worldfiles and flow tables RHESSysWorkflows needs a copy of RHESSys source code. RHESSysWorkflows also uses the new RHESSys ParamDB database and Python libraries to generate vegetation, soil, land use and other parameters needed by RHESSys. RHESSysWorkflows is only compatible with the pre-release version of RHESSys 5.16 and later versions of the code. At present, and for first-time users, the most reliable way to import ParamDB and RHESSys source code into your project is to download the code from GitHub using the **ImportRHESSysSource** tool. However, this tool is also capable of importing RHESSys source code stored on your computer. This allows you to import the code from a previous RHESSysWorkflows project; ParamDB is always downloaded from GitHub, even when RHESSys source code is imported from a local source.

In [ ]:
output = w.ImportRHESSysSource(w.sub_project_folder)
print 'Step Finished'

## Step 8 Import Climate Data

Because of the great variability of climate data formats, and the complexity of time-series workflows, we have chosen to focus development effort on RHESSysWorkflows toward making it easier to acquire and manipulate geospatial data required for building RHESSys work files and flow tables. This means that the modeler is responsible for building the climate data necessary for building RHESSys world files and performing model runs.

RHESSysWorkflows provides the **ImportClimateData** tool to import RHESSys climate data into your project:

In [ ]:
w.climate_data_fullpathname = w.sub_project_folder + datasource_folder + 'rhessys/clim'
w.ImportClimateData(w.sub_project_folder, w.climate_data_fullpathname)

## Step 9 Delineate Watershed
RHESSysWorkflows automates the process of delineating your study watershed based on the location of the streamflow gage registered in the workflow using EcohydroLib. As part of this process, many datasets needed by RHESSys will be derived from the DEM. To delineate the watershed we use the **DelineateWatershed** tool.

Here the **cell_size** parameter specifies the minimum size (in DEM cells) for subwatersheds generated by the GRASS command r.watershed.

The **area_estimate** parameter allows you to provide a guess of the area (in sq. km) of the delineated watershed. DelineateWatershed will report whether the watershed is within 20% of the area. You can view the delineated watershed in GRASS by displaying the raster map named basin. If the area or the shape of the delineated watershed differs greatly from what you expect, you may need to vary how DelineateWatershed snaps your streamflow gage onto the stream network. This is accomplished by either changing the -s (a.k.a. --streamThreshold) or stream threshold parameter and/or the -w (a.k.a. --streamWindow) parameter passed to r.findtheriver.

### Please note: If you used a different resource from HydroShare, you will need to modify the parameters.

In [ ]:
cell_size = 3000
area_estimate = 1.6
output = w.DelineateWatershed(w.sub_project_folder, cell_size, area_estimate)
#print(output)

## Step 10 Generate Patch Map

RHESSysWorkflows provides **GeneratePatchMap**, an automated tool for creating gridded and clumped patch maps. Gridded patch maps consist of a regular grid of the same resolution and extent as the DEM; clumped maps can be created using elevation or topographic wetness index rasters. Modelers can also use custom patch maps registered via EcohydroLib's RegisterRaster tool and imported into GRASS using ImportRasterMapIntoGRASS. 

In [ ]:
patch_options = " -p " + w.sub_project_folder + " -t grid"
w.GeneratePatchMapOptions(patch_options)

## Step 11 Process Soil Texture Map
Since we used EcohydroLib's SSURGO tools to generate percent sand and percent clay raster maps for our watershed, we can use the GRASS add-on r.soils.texture to generate USDA soil texture classes, for which RHESSys's ParamDB contains parameters. It is also possible to use custom soil maps (refer to the [Tutorial](https://github.com/selimnairb/RHESSysWorkflows#generating-rhessys-definitions-for-custom-soil-data)).

To generate our soil texture map in GRASS, as well as the corresponding RHESSys soil definition files, use the **GenerateSoilTextureMap** tool as follows:

In [ ]:
output = w.GenerateSoilTextureMap(w.sub_project_folder, options='--overwrite')
#print(output)

## Step 12 Generate derived landcover maps

Here we register leaf coverage.

In [ ]:
leafc_fullpathname_Src = w.sub_project_folder + datasource_folder + 'leafc.tif'
leafc_fullpathname_Dec = w.sub_project_folder + '/leafc_test.tif'
shutil.copy(leafc_fullpathname_Src, leafc_fullpathname_Dec)

options = " -p " + w.sub_project_folder + " -t leafc -r " + leafc_fullpathname_Dec + " --force"
#print(options)
w.RegisterRasterOptions(options)

Here we register rootdepth.

In [ ]:
rootdepth_fullpathname_Src = w.sub_project_folder + datasource_folder + 'rootdepth.tif'
rootdepth_fullpathname_Dec = w.sub_project_folder + '/rootdepth_test.tif'
shutil.copy(rootdepth_fullpathname_Src, rootdepth_fullpathname_Dec)

options = " -p " + w.sub_project_folder + " -t rootdepth -r " + rootdepth_fullpathname_Dec + " --force"
#print(options)
w.RegisterRasterOptions(options)

Now we generating these maps from your project directory into GRASS using the **ImportRasterMapIntoGRASS** tool.
### Please note: If you are using a different resource, edit these steps accordingly.

In [ ]:
w.ImportRasterMapIntoGRASS(w.sub_project_folder, 'landcover', 'nearest')
w.ImportRasterMapIntoGRASS(w.sub_project_folder, 'lai', 'nearest')
w.ImportRasterMapIntoGRASS(w.sub_project_folder, 'leafc', 'nearest')
w.ImportRasterMapIntoGRASS(w.sub_project_folder, 'rootdepth', 'nearest')
w.ImportRasterMapIntoGRASS(w.sub_project_folder, 'roof_connectivity', 'nearest')

## Step 13 Generate Rules and Reclassify

Next, we need to provide raster reclassification rules so that RHESSysWorkflows will know how to generate vegetation, land use, roads, impervious, and LAI maps from the landcover map.
To do this, we use the **RegisterLandcoverReclassRules** tool:

Here the -b (a.k.a. --buildPrototypeRules) option is used to generate prototype rules that we can edit as needed. To include LAI recode rules when registering prototype or existing rules, also specify the -l (a.k.a. --includeLaiRules) option.

In [ ]:
options = " -p " + w.sub_project_folder + " -b --includeLaiRules"
w.RegisterLandcoverReclassRulesOptions(options)

As run above, RegisterLandcoverReclassRules will result in the following four rules files being created in the rules directory of your project directory:

    stratum.rule
    landuse.rule
    impervious.rule
    road.rule
    lai-recode.rule


We will edit each of these files so that RHESSysWorkflows creates the correct maps for each map type. First, it's useful to understand a little about how these maps are made. RHESSysWorkflows uses GRASS's r.reclass command (r.recode for creating LAI maps), and so the rules files follow this format. <br/>
It's important to note that the landcover reclass rules for stratum and landuse must result in raster maps whose values labels match class names present in the RHESSys ParamDB database. Thus, be very careful in editing the righthand side of the expressions in your stratum and landuse reclass rules.

Once the landcover reclass rules are in place, it is very easy to generate the raster maps derived from the landcover data as well as the vegetation and land use definition files needed by RHESSys using ***GenerateLandcoverMapsOptions*** tool.

In [ ]:
#lc_options = " -p " + w.sub_project_folder + " -l "
lc_options = " -p " + w.sub_project_folder + " --makeLaiMap --overwrite"
#print(lc_options)
output = w.GenerateLandcoverMapsOptions(lc_options)
#print(output)

## Step 14 Generate World Template

Now we are almost ready to create the world file for our watershed. First we must create the template from which the world file will be created. To do this, we'll use the **GenerateWorldTemplate** tool. Fortunately this is very easy because the metadata store contains nearly all the information needed to create the template.

The -c (a.k.a. --climateStation) option specifies the climate station to associate with this world file template. The --aspectMinSlopeOne option is necessary work around limitations in the program used to create the world file when the input DEM has areas of low slope.

### Please note: If you used a different resource from HydroShare, you will need to modify the climate station parameter.

Sometimes, definition files (def) required modification and/or creation using your text editor (i.e. vi). This resource has the necessary def files prepared and we need to copy them into the appropriate directory. Generating these def files are explained in the references listed at the top and bottom of the notebook.

In [ ]:
dir_src = w.sub_project_folder + datasource_folder + 'rhessys/defs/'
dir_dst = w.sub_project_folder + '/rhessys/defs/'
for filename in os.listdir(dir_src):
    if filename.endswith('.def'):
        shutil.copy( dir_src + filename, dir_dst)

**When using custom datasets, we recommend printing the output here to identify issues.**

In [ ]:
world_options = " -p " + w.sub_project_folder + " -c dr5_composite3"
##world_options = " -p " + w.sub_project_folder
##rint(world_options)
output = w.GenerateWorldTemplateOptions(world_options)
print(output)

## Step 15 Create World File

Now use the **CreateWorldfile** tool to create a world file using this template.
Note: Output from CreateWorldfile has been piped to output variable. For debugging, uncomment print statement (i.e. remove #).

**When using custom datasets, we recommend printing the output here to identify issues.**

In [ ]:
output = w.CreateWorldfile(w.sub_project_folder)
#print(output)

## Step 16 Create Flow Table
As with worldfile creation, at this point in the workflow, RHESSysWorkflows's metadata store contains all the information needed to create a flow table using the createflowpaths (CF) RHESSys program. We'll use CreateFlowtable to create our flow table.

This will result in the creation of a flow table called world.flow in the flow directory of your rhessys directory. Now we have almost everything we need to run RHESSys simulations.

See the RHESSysWorkflows tutorial to learn how to [route surface flows for road pixels directly to the stream](https://github.com/selimnairb/RHESSysWorkflows#creating-the-flow-table) and [create surface flow tables using a roof connectivity map](https://github.com/selimnairb/RHESSysWorkflows#creating-a-surface-flow-table-using-a-roof-connectivity-map)


In [ ]:
##flow_options = " -p " + w.sub_project_folder + " --routeRoads --routeRoofs -v"
flow_options = " -p " + w.sub_project_folder
#print(flow_options)
output = w.CreateFlowtableOptions(flow_options)

## Step 17 Initializing vegetation carbon and nitrogen stores
RHESSys provides a program called LAIread to initialize vegetation carbon and nitrogen stores in your world file.

    Note, LAIread should only be used for RHESSys simulations with static vegetation (i.e. not dynamic vegetation mode enable via the -g command line option to RHESSys).

Initializing carbon and nitrogen stores is a multi-step process that involves running LAI read to generate a redefine worldfile, running a 3-day RHESSys simulation to incorporate the redefine worldfile, writing out a new worldfile with initialized vegetation carbon and nitrogen stores. RHESSysWorkflows automates all of these processes for you using the tool RunLAIRead, which can even figure out what date to start the 3-day RHESSys simulation on based on your climate data.

    In the current version of RHESSysWorkflows, RunLAIRead is only able to read simulation start dates from point time-series climate data. Users of ASCII or NetCDF gridded climate data must run LAI read by hand. The next release of RHESSysWorkflows will add support for gridded climate data to RunLAIRead.

To initialize vegetation carbon and nitrogen stores, LAIread relies on allometric relationships between leaf area and carbon and nitrogen mass in various plant tissues (e.g. leaf, stem, root). Consult the RHESSys wiki for more information on allometric relationships used by LAIread. These allometric parameters have not yet been added to the RHESSys ParamDB database proper. A default version of the parameters for RHESSys base vegetation classes is stored in the RHESSys ParamDB source coderepository. RHESSysWorkflows stores this file under the name allometric.txt in the allometry folder of the ParamDB of your rhessys/db folder. 


### When this step fails, a text editor like vi is required to modify the template file.
Messages such as:
#For patch 1067562 Could not find vegtype of -2147483648   
#FATAL ERROR: in construct_patch, landuse default ID 0 not found for patch 472919 
#FATAL ERROR: in construct_patch, landuse default ID 0 not found for patch 472919      
#FATAL ERROR: in construct_canopy_strata, canopy_strata default ID 0 not found. 
#For patch 1066207 Could not find vegtype of -2147483648 

<br/>
The tool below truncates these messages and reports -1 only.<br/>
To fix the template, follow these steps:<br/>
 (1) Click on jupyter icon, top of the web page.<br/>
 (2) Under **Files** tab control, click on **new** and start a new terminal.<br/>
 (3) Change your directory to the **w.sub_project_folder** location.<br/>
 i.e. **w.sub_project_folder** equals /home/jovyan/work/notebooks/data/dead_run_testV2/dead_run_testV2
 <br/>
 Run the command RunLAIRead.py -v -p= **w.sub_project_folder**                                                                                                          
 <br/>
 You will see messages similar to these: <br/>
 FATAL ERROR: in construct_patch, **landuse default ID 0** not found for patch 472919  <br/>
 FATAL ERROR: in construct_canopy_strata, **canopy_strata default ID 0** not found.  <br/>
 <br/>
 ### To fix these issues: <br/>
 cd **w.sub_project_folder**/rhessys/defs  <br/>
 cp landuse_undeveloped.def landuse_test.def <br/>
 cp stratum_deciduous.def stratum_test.def <br/>
 Using a text editor modify the template file. i.e. vi landuse_test.def <br/>
 vi landuse_test.def Change the number of the first line to macth the missing ID above. i.e. 0 Save File and Close <br/>
 vi stratum_test.def Change the number of the first line to macth the missing ID above. i.e. 0 Save File and Close <br/>
Change your directory to rhessys\templates<br/>
cd **w.sub_project_folder**/rhessys/templates  <br/>
 Using a text editor modify the template file. i.e. vi template<br/>
Modify the number of defs/landuse definitions from 3 to 4.<br/>
Modify the number of defs/stratum_ definitions from 3 to 4.<br/>
     <br/>
Then re-run these steps until you see no errors returned from RunLAIRead.py<br/>
CreateWorldfile.py -p **w.sub_project_folder**<br/>                                           
CreateFlowtable.py -p **w.sub_project_folder**<br/>
RunLAIRead.py -p **w.sub_project_folder** <br/>
 
 
 


In [ ]:
print(w.sub_project_folder)
output = w.RunLAIRead(w.sub_project_folder)
print(output)

output = w.CreateWorldfile(w.sub_project_folder)

## Step 18 Creating a RHESSys TEC file
We need one more thing before we can run our model, a TEC file. TEC stands for "temporal event controller". We use a TEC file to tell RHESSys to do things on at certain times during a simulation.

In [ ]:
w.RunCmd(w.sub_project_folder, 3)

## Step 19 Running RHESSys models
Once you have built a RHESSys model using RHESSysWorkflows, you are free to run your model manually. However this will not capture information about model runs in your project metadata. If you would like to record your runs in your project metadata, use the **RunModel** command.


Because the project metadata knows where RHESSys is installed in your project directory, you don't have to specify the full path of any of the RHESSys input files (e.g. world files, flow tables, TEC files, etc), you only need to specify the filenames. RunModel will echo RHESSys console outlet to the screen (if the -v or verbose option is specified as above), and will always save the same output into a file named 'rhessys.out' stored in the output folder for each model run. The output folder will be named based on the value you provide for the '-pre' or output prefix option. Model output is stored in the output directory of the rhessys directory of your project directory.

For more details on using RunModel see the RHESSysWorkflows [tutorial](https://github.com/selimnairb/RHESSysWorkflows#running-rhessys-models). <br/>
<br/>
## Depending on your simulation period, this step will take awhile.



In [ ]:
#options = " -p " + w.sub_project_folder + " -c dr5_composite3"
runmodel_options = " -p " + w.sub_project_folder + ' -d \"Test model run\" --basin -pre test -st 2008 1 1 1 -ed 2010 1 1 1 -w world_init -t tec_daily.txt -r world.flow -- -s 0.07041256017 133.552915269 1.81282283058 -sv 4.12459677088 78.3440566535 -gw 0.00736592779294 0.340346799457'

w.RunModelOptions(runmodel_options)

## Resources

* RHESSys
  * [Setup](http://fiesta.bren.ucsb.edu/~rhessys/setup/setup.html)
  * [Wiki](http://fiesta.bren.ucsb.edu/~rhessys/)
* Data
  * [HydroShare](https://www.hydroshare.org/)
  * [USGS Data and Tools](https://www.usgs.gov/products/data-and-tools/data-and-tools-topics)
  * [USDA Data gateway](https://gdg.sc.egov.usda.gov/)
  * [HydroTerre](http://hydroterre.psu.edu/)
  

## Funding
This work was supported by the following NSF grants

    Award no. 1239678 EAGER: Collaborative Research: Interoperability Testbed-Assessing a Layered Architecture for Integration of Existing Capabilities

    Award no. 0940841 DataNet Federation Consortium.

    Award no. 1148090 Collaborative Research: SI2-SSI: An Interactive Software Infrastructure for Sustaining Collaborative Innovation in the Hydrologic Sciences



## Credits

**Lawrence E. Band** lband@email.unc.edu <br/>
**Brian Miles** brian_miles@unc.edu <br/> 
**Laurence Lin** laurence.lin@icloud.com <br/>
**John Duncan** jon.m.duncan@gmail.com <br/>
**Lorne Leonard** lnl3@psu.edu <br/>
